# 🎯 Event Blinker AI Fine-Tuning Notebook

This notebook will help you fine-tune a language model for your Event Blinker application.

## What this notebook does:
1. **Checks GPU availability** (T4/V100/A100)
2. **Installs required libraries** (Unsloth, PEFT, transformers)
3. **Loads a base model** (Llama 3 8B or Mistral 7B)
4. **Fine-tunes on event assistant data**
5. **Exports to Ollama format (GGUF)**
6. **Downloads the model for local use**

---
**IMPORTANT:** Go to Runtime > Change runtime type > Select **T4 GPU** (free) or **A100** (Colab Pro)

---

## Step 1: Check GPU and Install Dependencies

In [ ]:
# Check GPU availability
import torch
print(f"\n🔍 Checking GPU...")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n✅ GPU is ready for fine-tuning!")
else:
    print("\n⚠️ No GPU detected! Go to Runtime > Change runtime type > Select T4 GPU")

In [ ]:
# Install Unsloth (fastest fine-tuning library - 2x faster than HuggingFace)
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install datasets huggingface_hub

print("\n✅ All dependencies installed!")

## Step 2: Load Base Model (Llama 3 8B with 4-bit Quantization)

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 2048  # Can go up to 8192 for longer contexts
dtype = None  # Auto-detect (float16 for T4, bfloat16 for A100)
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the base model
# Options:
# - "unsloth/llama-3-8b-bnb-4bit" (Llama 3 8B - RECOMMENDED)
# - "unsloth/mistral-7b-v0.3-bnb-4bit" (Mistral 7B)
# - "unsloth/Meta-Llama-3.1-8B-bnb-4bit" (Llama 3.1 8B)
# - "unsloth/gemma-2b-bnb-4bit" (Smaller, faster)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("\n✅ Model loaded successfully!")

In [ ]:
# Add LoRA adapters for efficient fine-tuning
# This only trains ~1-10% of parameters, saving memory and time

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank (higher = more capacity, but slower)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # 0 is optimized
    bias="none",
    use_gradient_checkpointing="unsloth",  # 30% less memory
    random_state=3407,
)

print("\n✅ LoRA adapters added!")
print(f"Trainable parameters: {model.print_trainable_parameters()}")

## Step 3: Prepare Training Data for Event Blinker

### Fine-tuning Data Categories:
1. **Event Information** - Questions about event details, time, location, price
2. **Navigation/Directions** - How to get to events, parking info
3. **Booking/Registration** - How to register, ticket purchase
4. **General Assistance** - Ride-sharing, food options, accessibility
5. **Organizer Help** - Creating events, managing attendees

In [ ]:
# Event Blinker Training Dataset
# This is your custom training data for the event assistant

event_blinker_data = [
    # === EVENT INFORMATION ===
    {
        "instruction": "What time does the event start?",
        "context": "Event: Summer Music Festival. Location: Central Park. Starts at: 2024-07-15 18:00. Price: $50.",
        "response": "The Summer Music Festival starts at 6:00 PM on July 15th, 2024. Make sure to arrive a bit early to find good spots!"
    },
    {
        "instruction": "Where is this event located?",
        "context": "Event: Tech Meetup. Location: Innovation Hub, 123 Tech Street. Starts at: 2024-08-20 10:00.",
        "response": "The Tech Meetup is at Innovation Hub, located at 123 Tech Street. It's in the downtown tech district."
    },
    {
        "instruction": "How much are the tickets?",
        "context": "Event: Comedy Night. Location: Laugh Factory. Price: $25. Capacity: 200.",
        "response": "Tickets for Comedy Night are $25 each. The venue has a capacity of 200, so I'd recommend booking early!"
    },
    {
        "instruction": "Is this event free?",
        "context": "Event: Community Cleanup Day. Location: Riverside Park. Price: free.",
        "response": "Yes! Community Cleanup Day is completely free to attend. Just show up at Riverside Park and join the volunteers!"
    },
    {
        "instruction": "What is this event about?",
        "context": "Event: AI Workshop. Description: Learn the basics of machine learning, neural networks, and build your first AI model.",
        "response": "The AI Workshop is designed to teach you machine learning basics, how neural networks work, and guide you through building your first AI model. Perfect for beginners!"
    },
    {
        "instruction": "How many people can attend?",
        "context": "Event: Startup Pitch Night. Capacity: 150. Current attendance: 89.",
        "response": "Startup Pitch Night has a capacity of 150 attendees. Currently, 89 spots are taken, so 61 spots are still available. Register soon!"
    },

    # === NAVIGATION & DIRECTIONS ===
    {
        "instruction": "How do I get to the event?",
        "context": "Event: Art Exhibition. Location: Modern Art Gallery, 456 Art Avenue.",
        "response": "The Art Exhibition is at Modern Art Gallery, 456 Art Avenue. You can use the Event Blinker app's map feature to get turn-by-turn directions. The gallery is accessible by both public transit and car."
    },
    {
        "instruction": "Is there parking available?",
        "context": "Event: Food Festival. Location: City Convention Center.",
        "response": "The City Convention Center has a large parking garage attached. Parking typically costs $10-15 for the day. There's also street parking nearby, but it fills up fast during events."
    },
    {
        "instruction": "Can I take public transport?",
        "context": "Event: Marathon Race. Location: Downtown Sports Complex.",
        "response": "Yes! The Downtown Sports Complex is well-served by public transit. Take the Blue Line to Central Station, then it's a 5-minute walk. Bus routes 12 and 34 also stop nearby."
    },
    {
        "instruction": "What's the best route from the airport?",
        "context": "Event: International Conference. Location: Grand Hotel Conference Center.",
        "response": "From the airport, take the Airport Express train to Central Station (30 mins), then a taxi or rideshare to Grand Hotel (10 mins). Total journey is about 45 minutes. Alternatively, taxis from the airport cost around $40-50."
    },

    # === BOOKING & REGISTRATION ===
    {
        "instruction": "How do I register for this event?",
        "context": "Event: Yoga Retreat. Registration: Open.",
        "response": "To register for the Yoga Retreat, tap the 'Register' or 'Book Now' button on the event page in the app. You'll be guided through the registration process. Make sure you're logged in first!"
    },
    {
        "instruction": "Can I get a refund?",
        "context": "Event: Photography Workshop. Price: $75.",
        "response": "Refund policies vary by event organizer. For the Photography Workshop, I'd recommend contacting the organizer directly through the chat feature. Generally, refunds are available up to 48 hours before the event."
    },
    {
        "instruction": "Can I bring a friend?",
        "context": "Event: Movie Night. Capacity: 100. Current: 45.",
        "response": "Absolutely! Movie Night has plenty of space - 55 spots are still available. Your friend will need to register separately through the app, or you can purchase multiple tickets if that option is available."
    },

    # === RIDE SHARING ===
    {
        "instruction": "Are there any ride shares to this event?",
        "context": "Event: Mountain Hiking Trip. Location: Trailhead Parking Lot.",
        "response": "Check the 'Rides' tab on the event page to see if other attendees are offering ride shares. You can also offer your own ride if you're driving! It's a great way to meet fellow hikers and split costs."
    },
    {
        "instruction": "How does ride sharing work?",
        "context": "General question about Event Blinker features.",
        "response": "Event Blinker's ride sharing lets you connect with other attendees! Drivers can offer seats, and riders can request rides. Simply go to any event, tap 'Rides', and either offer a ride or browse available ones. You can chat with drivers before confirming."
    },

    # === GENERAL ASSISTANCE ===
    {
        "instruction": "What should I bring?",
        "context": "Event: Beach Party. Location: Sunset Beach.",
        "response": "For the Beach Party at Sunset Beach, bring: sunscreen, towel, swimwear, water bottle, and maybe a change of clothes. The organizer may provide some refreshments, but check the event description for details!"
    },
    {
        "instruction": "Is the venue wheelchair accessible?",
        "context": "Event: Classical Concert. Location: Symphony Hall.",
        "response": "Symphony Hall is fully wheelchair accessible with ramps, elevators, and accessible seating areas. Contact the venue or organizer in advance to reserve accessible seating."
    },
    {
        "instruction": "Will there be food?",
        "context": "Event: All-Day Conference. Duration: 8 hours.",
        "response": "For an all-day conference, catering is usually provided (coffee breaks, lunch). Check the event description for specific details. If not mentioned, there are usually cafeterias or restaurants nearby."
    },
    {
        "instruction": "What's the dress code?",
        "context": "Event: Gala Dinner. Location: Luxury Hotel Ballroom.",
        "response": "For a Gala Dinner at a Luxury Hotel Ballroom, the dress code is typically formal/black tie. Men should wear suits or tuxedos, women should wear evening dresses or formal attire."
    },
    {
        "instruction": "Can I bring my kids?",
        "context": "Event: Family Fun Day. Target audience: Families.",
        "response": "Absolutely! Family Fun Day is designed for families and kids are very welcome! There will likely be activities for all ages. Check the event description for specific kids' activities."
    },

    # === ORGANIZER HELP ===
    {
        "instruction": "How do I create an event?",
        "context": "User is an organizer.",
        "response": "To create an event: 1) Log into the web portal at your-event-blinker-url.com, 2) Click 'Create Event', 3) Fill in title, description, date/time, location, and price, 4) Add images, 5) Click 'Publish'. Your event will appear on the map immediately!"
    },
    {
        "instruction": "How do I see who's attending my event?",
        "context": "User is an organizer checking their event.",
        "response": "Go to your organizer dashboard and select the event. You'll see the 'Attendees' tab with a full list of registered attendees, their check-in status, and contact info (if provided)."
    },
    {
        "instruction": "How do I update my event details?",
        "context": "User wants to edit their event.",
        "response": "To update your event: 1) Go to your organizer dashboard, 2) Find your event and click 'Edit', 3) Make your changes, 4) Click 'Save'. Attendees will be notified of major changes automatically."
    },

    # === EDGE CASES & UNKNOWN ===
    {
        "instruction": "What's the weather going to be like?",
        "context": "Event: Outdoor Concert.",
        "response": "I don't have access to weather forecasts, but I recommend checking your weather app closer to the event date. For outdoor events, it's always good to bring layers and check if the organizer has a rain policy!"
    },
    {
        "instruction": "Can you call the organizer for me?",
        "context": "User request for phone call.",
        "response": "I can't make phone calls, but you can contact the organizer directly through the event's chat feature in the app. Tap on the event, then 'Contact Organizer' or 'Chat' to send them a message."
    },
    {
        "instruction": "What if the event is cancelled?",
        "context": "General question.",
        "response": "If an event is cancelled, you should receive a notification from Event Blinker. Refund policies depend on the organizer - check your booking confirmation or contact them directly. Most organizers offer full refunds for cancellations."
    },

    # === CONVERSATIONAL & GREETINGS ===
    {
        "instruction": "Hello!",
        "context": "General greeting.",
        "response": "Hello! 👋 I'm your Event Blinker assistant. I can help you with event information, directions, registration, and more. What would you like to know?"
    },
    {
        "instruction": "Thank you!",
        "context": "User expressing gratitude.",
        "response": "You're welcome! Enjoy your event! 🎉 If you have any more questions, just ask. Have a great time!"
    },
    {
        "instruction": "What can you help me with?",
        "context": "User asking about capabilities.",
        "response": "I can help you with: \n• Event details (time, location, price)\n• Directions and parking info\n• Registration and booking\n• Finding ride shares\n• General event questions\n\nJust ask about any event you're interested in!"
    }
]

print(f"\n✅ Created {len(event_blinker_data)} training examples!")

In [ ]:
# Format data for training (Alpaca-style prompt format)
from datasets import Dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with context that provides further information. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Context:
{context}

### Response:
{response}"""

EOS_TOKEN = tokenizer.eos_token

def format_prompt(example):
    return {
        "text": alpaca_prompt.format(
            instruction=example["instruction"],
            context=example["context"],
            response=example["response"]
        ) + EOS_TOKEN
    }

# Create dataset
dataset = Dataset.from_list(event_blinker_data)
dataset = dataset.map(format_prompt)

print("\n✅ Dataset formatted!")
print(f"\n📝 Sample training example:\n")
print(dataset[0]["text"][:500] + "...")

## Step 4: Fine-Tune the Model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can be True for faster training on short examples
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # Increase for more epochs (e.g., 200 for production)
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

print("\n🚀 Starting training...\n")

In [ ]:
# Train the model!
trainer_stats = trainer.train()

print("\n✅ Training complete!")
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"Training loss: {trainer_stats.metrics['train_loss']:.4f}")

## Step 5: Test the Fine-Tuned Model

In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model)

test_prompts = [
    {"instruction": "What time does the event start?", "context": "Event: Jazz Night. Starts at: 8:00 PM. Location: Blue Note Cafe."},
    {"instruction": "Is there parking available?", "context": "Event: Food Truck Festival. Location: Downtown Plaza."},
    {"instruction": "How do I register?", "context": "Event: Coding Bootcamp. Price: $200."},
]

for test in test_prompts:
    prompt = alpaca_prompt.format(
        instruction=test["instruction"],
        context=test["context"],
        response=""
    )
    
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the response part
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    
    print(f"\n❓ {test['instruction']}")
    print(f"📍 Context: {test['context']}")
    print(f"🤖 Response: {response}")
    print("-" * 60)

## Step 6: Export Model for Ollama (GGUF Format)

In [ ]:
# Save the LoRA adapters first
model.save_pretrained("event_blinker_lora")
tokenizer.save_pretrained("event_blinker_lora")
print("\n✅ LoRA adapters saved!")

In [ ]:
# Export to GGUF format for Ollama
# Quantization options:
# - q4_k_m: Best balance of size and quality (RECOMMENDED)
# - q5_k_m: Slightly larger, slightly better quality
# - q8_0: Largest, best quality

model.save_pretrained_gguf(
    "event_blinker_model",
    tokenizer,
    quantization_method="q4_k_m"
)

print("\n✅ Model exported to GGUF format!")
print("\nFile: event_blinker_model-unsloth.Q4_K_M.gguf")

In [ ]:
# Create Modelfile for Ollama
modelfile_content = '''FROM ./event_blinker_model-unsloth.Q4_K_M.gguf

TEMPLATE """Below is an instruction that describes a task, paired with context that provides further information. Write a response that appropriately completes the request.

### Instruction:
{{ .Prompt }}

### Context:
{{ .Context }}

### Response:
"""

SYSTEM "You are an event assistant for Event Blinker app. Be concise, friendly, and helpful. Help users with event information, directions, registration, and general questions."

PARAMETER temperature 0.7
PARAMETER top_p 0.9
PARAMETER stop "### Instruction:"
PARAMETER stop "### Context:"
'''

with open("Modelfile", "w") as f:
    f.write(modelfile_content)

print("\n✅ Modelfile created!")
print("\n📝 Modelfile contents:")
print(modelfile_content)

In [ ]:
# Zip the model files for download
!zip -r event_blinker_ollama_model.zip event_blinker_model-unsloth.Q4_K_M.gguf Modelfile

print("\n✅ Model files zipped!")
print("\n📦 Download: event_blinker_ollama_model.zip")

In [ ]:
# Download the model
from google.colab import files
files.download("event_blinker_ollama_model.zip")

print("\n🎉 Download started! After downloading:")
print("")
print("1. Unzip the file")
print("2. Move files to your Event Blinker project")
print("3. Run: ollama create event-blinker -f Modelfile")
print("4. Update your .env: OLLAMA_MODEL=event-blinker")
print("5. Restart your AI service!")

## 🎉 Complete!

### How to Use the Fine-Tuned Model:

1. **Download** the `event_blinker_ollama_model.zip` file
2. **Unzip** and copy files to your project
3. **Create the Ollama model**:
   ```bash
   ollama create event-blinker -f Modelfile
   ```
4. **Update your AI service .env**:
   ```
   OLLAMA_MODEL=event-blinker
   ```
5. **Restart the AI service** and test!

---

### Tips for Better Fine-Tuning:

1. **More data = Better results**: Add more training examples (100-500 recommended)
2. **More epochs**: Increase `max_steps` to 100-200 for better learning
3. **Domain-specific data**: Add real event Q&As from your users
4. **Categories to add**:
   - Safety/emergency info
   - Weather-related questions
   - Group bookings
   - Vendor/sponsor questions
   - Multi-day event handling